# Perceptron applied to the Iris dataset

**Reading the dataset:** $\;$ we also check that the data matrix and labels have the right number of rows and columns

In [1]:
import numpy as np; from sklearn.datasets import load_digits
iris = load_digits(); X = iris.data.astype(np.float16);
y = iris.target.astype(np.uint).reshape(-1, 1);
print(X.shape, y.shape, "\n", np.hstack([X, y])[:5, :])

(1797, 64) (1797, 1) 
 [[ 0.  0.  5. 13.  9.  1.  0.  0.  0.  0. 13. 15. 10. 15.  5.  0.  0.  3.
  15.  2.  0. 11.  8.  0.  0.  4. 12.  0.  0.  8.  8.  0.  0.  5.  8.  0.
   0.  9.  8.  0.  0.  4. 11.  0.  1. 12.  7.  0.  0.  2. 14.  5. 10. 12.
   0.  0.  0.  0.  6. 13. 10.  0.  0.  0.  0.]
 [ 0.  0.  0. 12. 13.  5.  0.  0.  0.  0.  0. 11. 16.  9.  0.  0.  0.  0.
   3. 15. 16.  6.  0.  0.  0.  7. 15. 16. 16.  2.  0.  0.  0.  0.  1. 16.
  16.  3.  0.  0.  0.  0.  1. 16. 16.  6.  0.  0.  0.  0.  1. 16. 16.  6.
   0.  0.  0.  0.  0. 11. 16. 10.  0.  0.  1.]
 [ 0.  0.  0.  4. 15. 12.  0.  0.  0.  0.  3. 16. 15. 14.  0.  0.  0.  0.
   8. 13.  8. 16.  0.  0.  0.  0.  1.  6. 15. 11.  0.  0.  0.  1.  8. 13.
  15.  1.  0.  0.  0.  9. 16. 16.  5.  0.  0.  0.  0.  3. 13. 16. 16. 11.
   5.  0.  0.  0.  0.  3. 11. 16.  9.  0.  2.]
 [ 0.  0.  7. 15. 13.  1.  0.  0.  0.  8. 13.  6. 15.  4.  0.  0.  0.  2.
   1. 13. 13.  0.  0.  0.  0.  0.  2. 15. 11.  1.  0.  0.  0.  0.  0.  1.
  12. 12.  1.  0.  0. 

**Dataset partition:** $\;$ We create a split of the Iris dataset with $20\%$ of data for test and the rest for training, previously shuffling the data according to a given seed provided by a random number generator. Here, as in all code that includes randomness (which requires generating random numbers), it is convenient to fix said seed to be able to reproduce experiments with accuracy.

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)
print(X_train.shape, X_test.shape)

(1437, 64) (360, 64)


**Perceptron implementation:** $\;$ returns weights in homogeneous notation, $\mathbf{W}\in\mathbb{R}^{(1+D)\times C};\;$ also the number of errors and iterations executed

In [3]:
def perceptron(X, y, b=0.1, a=1.0, K=200):
    N, D = X.shape; Y = np.unique(y); C = Y.size; W = np.zeros((1+D, C))
    for k in range(1, K+1):
        E = 0
        for n in range(N):
            xn = np.array([1, *X[n, :]])
            cn = np.squeeze(np.where(Y==y[n]))
            gn = W[:,cn].T @ xn; err = False
            for c in np.arange(C):
                if c != cn and W[:,c].T @ xn + b >= gn:
                    W[:, c] = W[:, c] - a*xn; err = True
            if err:
                W[:, cn] = W[:, cn] + a*xn; E = E + 1
        if E == 0:
            break;
    return W, E, k

**Learning a (linear) classifier with Perceptron:** $\;$ Perceptron minimizes the number of training errors (with margin $b$)
$$\mathbf{W}^*=\operatorname*{argmin}_{\mathbf{W}=(\boldsymbol{w}_1,\dotsc,\boldsymbol{w}_C)}\sum_n\;\mathbb{ I}\biggl(\max_{c\neq y_n}\;\boldsymbol{w}_c^t\boldsymbol{x}_n+b \;>\; \boldsymbol{w}_{y_n}^t\boldsymbol{ x}_n\biggr)$$

In [4]:
W, E, k = perceptron(X_train, y_train)
print("Number of iterations executed: ", k)
print("Number of training errors: ", E)
print("Weight vectors of the classes (in columns and with homogeneous notation):\n", W);

Number of iterations executed:  106
Number of training errors:  0
Weight vectors of the classes (in columns and with homogeneous notation):
 [[-1.420e+02 -2.120e+02 -1.420e+02 -1.430e+02 -1.040e+02 -1.480e+02
  -1.390e+02 -1.410e+02 -1.450e+02 -1.910e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [-1.700e+01 -2.600e+01  3.300e+01  5.000e+01 -3.500e+01 -1.300e+01
  -4.200e+01  4.200e+01 -5.700e+01 -1.720e+02]
 [-8.910e+02 -1.029e+03 -9.100e+02 -9.130e+02 -8.400e+02 -5.020e+02
  -1.030e+03 -8.650e+02 -8.160e+02 -9.250e+02]
 [-1.639e+03 -1.522e+03 -1.635e+03 -1.680e+03 -2.119e+03 -1.819e+03
  -1.746e+03 -1.539e+03 -1.805e+03 -1.457e+03]
 [-1.708e+03 -2.382e+03 -1.691e+03 -1.321e+03 -1.808e+03 -1.673e+03
  -1.757e+03 -1.597e+03 -1.682e+03 -1.599e+03]
 [-1.084e+03 -4.850e+02 -9.800e+02 -9.290e+02 -1.280e+03 -6.760e+02
  -1.110e+03 -8.550e+02 -1.067e+03 -1.035e+03]
 [-3.240e+02 -2.790e+02 -2.640e+02 -2.130e+02 -3.290e+

**Calculation of test error rate:**

In [5]:
X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])
y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)
err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
print(f"Error rate on test: {err_test:.1%}")

Error rate on test: 4.4%


**Margin adjustment:** $\;$ experiment to learn a value of $b$

In [6]:
for b in (.0, .01, .1, 10, 100):
    W, E, k = perceptron(X_train, y_train, b=b, K=1000)
    print(b, E, k)

0.0 0 106
0.01 0 106
0.1 0 106
10 0 140
100 0 97
